In [2]:
import cv2
import mediapipe as mp
import pandas as pd
import re

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()

In [3]:
# Regular expression pattern to extract coordinates
pattern = r'x: ([-+]?[0-9]*\.?[0-9]+)\ny: ([-+]?[0-9]*\.?[0-9]+)\nz: ([-+]?[0-9]*\.?[0-9]+)'

def get_coordinates(input_string):
    # Extract coordinates using regex
    matches = re.findall(pattern, input_string)

    # Convert the extracted coordinates to a list of dictionaries
    coordinates = [{'x': float(match[0]), 'y': float(match[1]), 'z': float(match[2])} for match in matches]
    return coordinates

In [4]:
import os
path = os.getcwd()

In [11]:
def extract_video(filename):
    global df
    video = f'{path}/{filename}.mp4'
    # For webcam input:
    cap = cv2.VideoCapture(video)
    results = None
    with mp_holistic.Holistic(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame. Or video finished. ")
                # If loading a video, use 'break' instead of 'continue'.
                break

            results = hands.process(image)

            # # Draw the hand annotations on the image.
            image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if (results.right_hand_landmarks):
                array = get_coordinates(str(results.right_hand_landmarks.landmark))
                new_row = {}
                for i in range(len(array)):
                    new_row[f'x_right_hand_{i}'] = array[i]["x"]
                    new_row[f'y_right_hand_{i}'] = array[i]["y"]

                
                df = df.append(new_row, ignore_index=True)

                mp_drawing.draw_landmarks(
                    image,
                    results.right_hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
            
            if (results.left_hand_landmarks):
                array = get_coordinates(str(results.left_hand_landmarks.landmark))
                new_row = {}
                for i in range(len(array)):
                    new_row[f'x_left_hand_{i}'] = array[i]["x"]
                    new_row[f'y_left_hand_{i}'] = array[i]["y"]
                
                df = df.append(new_row, ignore_index=True)

                mp_drawing.draw_landmarks(
                    image,
                    results.left_hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
              break

    cap.release()
    cv2.destroyAllWindows()

In [12]:
filename = "Hello"
extract_video(filename)
df.to_parquet(f'{filename}.parquet')

Ignoring empty camera frame. Or video finished. 


In [7]:
pd.read_parquet(f'{filename}.parquet', engine='pyarrow')

,x_right_hand_0,x_right_hand_1,x_right_hand_10,x_right_hand_11,x_right_hand_12,x_right_hand_13,x_right_hand_14,x_right_hand_15,x_right_hand_16,x_right_hand_17,...,y_right_hand_19,y_right_hand_2,y_right_hand_20,y_right_hand_3,y_right_hand_4,y_right_hand_5,y_right_hand_6,y_right_hand_7,y_right_hand_8,y_right_hand_9
0,0.289231,0.275988,0.226214,0.233582,0.245503,0.238860,0.224984,0.232368,0.243127,0.237180,...,0.627634,0.625707,0.633307,0.620097,0.618336,0.564095,0.592634,0.617303,0.628569,0.563830
1,0.267388,0.280037,0.258154,0.265950,0.270221,0.252380,0.251188,0.257966,0.261604,0.245566,...,0.452443,0.518865,0.467174,0.502569,0.487845,0.460151,0.426198,0.434356,0.452609,0.461078
2,0.281588,0.299793,0.289746,0.293492,0.296570,0.278378,0.280008,0.283870,0.287480,0.268757,...,0.340204,0.425922,0.334128,0.404926,0.384692,0.365895,0.325989,0.316664,0.313911,0.365740
3,0.297870,0.319153,0.325865,0.331043,0.335532,0.308340,0.314077,0.319502,0.324481,0.296555,...,0.245889,0.357535,0.238318,0.336018,0.315779,0.290647,0.250747,0.237520,0.229839,0.284608
4,0.316143,0.339687,0.359756,0.365969,0.371405,0.336600,0.348206,0.354895,0.360675,0.324365,...,0.198151,0.310091,0.186577,0.287591,0.272078,0.255105,0.217655,0.199265,0.183667,0.243683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,0.255669,0.272621,0.237685,0.232878,0.228153,0.234001,0.226601,0.223717,0.221144,0.224135,...,0.158863,0.201940,0.140941,0.166473,0.137736,0.168146,0.129767,0.107209,0.086644,0.173149
256,0.251066,0.266520,0.229510,0.224505,0.219736,0.225991,0.219550,0.216431,0.213751,0.216865,...,0.173298,0.213073,0.156891,0.178311,0.149949,0.179419,0.142366,0.121349,0.102358,0.184897
257,0.244841,0.257820,0.217033,0.211487,0.207012,0.216503,0.208196,0.203867,0.201033,0.208307,...,0.203780,0.232583,0.189126,0.199616,0.173802,0.201886,0.168769,0.150526,0.135047,0.209122
258,0.237213,0.247475,0.200861,0.193864,0.187533,0.204367,0.193683,0.187539,0.182434,0.198187,...,0.251657,0.263955,0.243984,0.234964,0.213426,0.237140,0.209530,0.197049,0.188401,0.247198
